In [48]:
import sqlite3
from pyowm import OWM
import os
import requests
from dotenv import load_dotenv
import datetime

In [49]:
# Load OWM API key from .env file
load_dotenv()
api_key = os.getenv('OWM_API_KEY')

In [50]:
# Create a SQLite database with a table for weather data
DB_FILE = 'weatherdata.db'
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

In [51]:
# Create the table if it doesn't exist, runs only once
sql_create_table ='''
    CREATE TABLE IF NOT EXISTS hbg_weather (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT NOT NULL,
        status TEXT NOT NULL,
        temperature REAL NOT NULL,
        humidity REAL NOT NULL,
        wind REAL NOT NULL,
        clouds REAL NOT NULL
    );
'''
cursor.execute(sql_create_table)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
# pyowm https://github.com/csparpa/pyowm
def get_weather_data(place='Helsingborg,SE'):
    owm = OWM(api_key)
    mgr = owm.weather_manager()

    # Current weather in Helsingborg
    observation = mgr.weather_at_place(place)
    w = observation.weather

    status = w.detailed_status
    temperature = w.temperature('celsius')
    wind = w.wind()
    humidity = w.humidity
    clouds = w.clouds
    timestamp = datetime.datetime.now().isoformat()

    return (timestamp, status, temperature['temp'], humidity, wind.get('speed', 0), clouds)

In [ ]:
# Insert weather data into the table
def insert_weather(data):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()    
     
    sql_insert = '''
        INSERT INTO hbg_weather (timestamp, status, temperature, humidity, wind, clouds)
        VALUES (?, ?, ?, ?, ?, ?);
    '''
    try:
        cursor.execute(sql_insert, data)
        conn.commit()
        print("Data inserted successfully.")
    except sqlite3.Error as e:
        print(f"Error inserting data: {e}")
        # Log the error here
    finally:
        conn.close()

# Get the latest weather data and insert it into the database
weather_data_to_insert = get_weather_data()
insert_weather(weather_data_to_insert)

Data inserted successfully.


In [76]:
# Check that data has been inserted
def select_all_data():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM hbg_weather;")
    
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
        
    conn.close()

select_all_data()

(1, '2025-08-28T19:58:16.748380', 'light rain', 18.87, 82.0, 4.48, 100.0)
